In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Start

In [ ]:
import sys
sys.path.append("..") 

from nb_004 import *

In [ ]:
DATA_PATH = Path('../data')
PATH = DATA_PATH/'caltech101'

data_mean,data_std = map(tensor, ([0.5355,0.5430,0.5280], [0.2909,0.2788,0.2979]))

classes = ['airplanes','Motorbikes','Faces','watch','Leopards']
np.random.seed(42)
train_ds,valid_ds = FilesDataset.from_folder(PATH, test_pct=0.2, classes=classes)
classes = train_ds.classes
c = len(classes); c

In [ ]:
sz = 224
norm = normalize_tfm(mean=data_mean,std=data_std)
denorm = partial(denormalize,mean=data_mean,std=data_std)

trn_tfms = [squish_tfm(scale=(0.75,1.33), row_pct=(0,1.), col_pct=(0,1.)),
            zoom_tfm(scale=(0.8,1.), row_pct=(0,1.), col_pct=(0,1.)),
            flip_lr_tfm(p=0.5),
            crop_tfm(size=sz),
            norm]
val_tfms = [crop_tfm(size=sz), norm]

train_ds = TfmDataset(train_ds, trn_tfms, size=224, do_crop=True)
valid_ds = TfmDataset(valid_ds, val_tfms, size=224, do_crop=True)

data = DataBunch(train_ds, valid_ds, bs=64, num_workers=8)

## Train

In [ ]:
def simple_cnn(n_classes, actns, kernel_szs, strides, bn=False):
    kernel_szs = listify(kernel_szs, len(actns)-1)
    strides    = listify(strides   , len(actns)-1)
    layers = [conv2_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i], bn=bn)
        for i in range(len(strides))]
    layers += [PoolFlatten(), nn.Linear(actns[-1], n_classes)]
    return nn.Sequential(*layers)

def conv2_relu(nif, nof, ks, stride, bn=False):
    layers = [nn.Conv2d(nif, nof, ks, stride, padding=ks//2), nn.ReLU()]
    if bn: layers.append(nn.BatchNorm2d(nof))
    return nn.Sequential(*layers)

In [ ]:
def get_model(): return simple_cnn(c, [3,16,16,16], 3, 2, bn=True)

In [ ]:
def fit_one_cycle(learn:Learner, max_lr:float, cyc_len:int, moms=(0.95,0.85), div_factor:float=10.,
                 pct_end:float=0.1, wd:float=0.):
    "Fits a model following the 1cycle policy"
    cbs = [OneCycleScheduler(learn, max_lr, cyc_len, moms, div_factor, pct_end)]
    learn.fit(cyc_len, max_lr/div_factor, wd=wd, callbacks=cbs)

In [ ]:
model = get_model()
learn = Learner(data, model)
learn.loss_fn = F.cross_entropy
learn.metrics = [accuracy]
learn.opt_fn = partial(optim.Adam, betas=(0.95,0.99))
learn.true_wd = True

In [ ]:
fit_one_cycle(learn, 1e-3, 1, wd=0.1)

In [ ]:
model[0][2].running_mean

In [ ]:
model.eval()
with torch.no_grad():
    *val_metrics,nums = zip(*[loss_batch(model, xb, yb, learn.loss_fn, metrics=learn.metrics)
                                for xb,yb in learn.data.valid_dl])
    val_metrics = [np.sum(np.multiply(val,nums)) / np.sum(nums) for val in val_metrics]

val_metrics

In [ ]:
torch.save(model.state_dict(),PATH/'model1.pt')

## Eval

In [ ]:
model1 = get_model()
learn1 = Learner(data, model1)
learn1.loss_fn = F.cross_entropy
learn1.metrics = [accuracy]
learn1.opt_fn = partial(optim.Adam, betas=(0.95,0.99))
learn1.true_wd = True

In [ ]:
model1.load_state_dict(torch.load(PATH/'model1.pt'))

In [ ]:
model1.eval()
with torch.no_grad():
    *val_metrics,nums = zip(*[loss_batch(model1, xb, yb, learn1.loss_fn, metrics=learn1.metrics)
                                for xb,yb in learn1.data.valid_dl])
    val_metrics = [np.sum(np.multiply(val,nums)) / np.sum(nums) for val in val_metrics]

val_metrics

In [ ]:
model1[0][2].running_mean

In [ ]:
model1[0][2].running_mean